# Predict Median Household Income

This notebook shows how to use supervised machine learning to predict median household income for block groups in California using American Community Survey 2017-2021 5-year data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import joblib

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
census_data = pd.read_csv('census_data.csv')
census_data.head(3)

## Define Label and Features

The label is median household income, which is missing for some block groups. A predictive model could be valuable for filling those missing values.

For this analysis, the label and all predictive features are numeric (not categorical).

In [ ]:
label = ['B19013_001E_Estimate_Median_household_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars)']
features = [
    'B01002_001E_Estimate_Median_age_--_Total',
    'B01001_026E_Estimate_Total_Female_P',
    'B03002_003E_Estimate_Total_Not_Hispanic_or_Latino_White_alone_P',
    'B03002_004E_Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone_P',
    'B03002_005E_Estimate_Total_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone_P',
    'B03002_006E_Estimate_Total_Not_Hispanic_or_Latino_Asian_alone_P',
    'B03002_007E_Estimate_Total_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone_P',
    'B03002_008E_Estimate_Total_Not_Hispanic_or_Latino_Some_other_race_alone_P',
    'B03002_009E_Estimate_Total_Not_Hispanic_or_Latino_Two_or_more_races_P',
    'B03002_012E_Estimate_Total_Hispanic_or_Latino_P',
    'B99162_002E_Estimate_Total_Speak_only_English_P',
    'C17002_002E_Estimate_Total_Under_.50_P',
    'C17002_003E_Estimate_Total_.50_to_.99_P',
    'C17002_004E_Estimate_Total_1.00_to_1.24_P',
    'C17002_005E_Estimate_Total_1.25_to_1.49_P',
    'C17002_006E_Estimate_Total_1.50_to_1.84_P',
    'C17002_007E_Estimate_Total_1.85_to_1.99_P',
    'C17002_008E_Estimate_Total_2.00_and_over_P',
    'B07201_002E_Estimate_Total_Same_house_1_year_ago_P',
    'B08301_002E_Estimate_Total_Car,_truck,_or_van_P',
    'B08301_010E_Estimate_Total_Public_transportation_(excluding_taxicab)_P',
    'B08301_019E_Estimate_Total_Walked_P',
    'B08301_021E_Estimate_Total_Worked_from_home_P',
    'B25003_003E_Estimate_Total_Renter_occupied_P'
]

## Explore the Data

First, we will look at summary statistics and counts of missing data for the label and features.

In [ ]:
census_data[label + features].describe().apply(lambda s: s.apply('{0:.2f}'.format))

In [ ]:
census_data[census_data[label[0]] == -666666666][label[0]].value_counts()

In [ ]:
census_data[census_data[features[0]] == -666666666][features[0]].value_counts()

In [ ]:
census_data[label + features].isna().sum()

Median income and median age both have missing values indicated by the value '-666666666'. Additionally, the other features have null values constituting a small percent of the data. We will address the missing data before continuing with exploratory data analysis.

For the features, we will replace missing values with the median value of the feature (assuming some features may be skewed, and therefore median may be more of a "typical" value than the mean).

In [ ]:
census_data_cleaned = census_data.copy()
for i in features:
    census_data_cleaned[i] = census_data_cleaned[i].fillna(census_data_cleaned[i].median())
    census_data_cleaned.loc[census_data_cleaned[i] == -666666666, i] = census_data_cleaned[i].median()

For the label, we will pull out the records with a missing median income for use later as a set of new data to predict.

In [ ]:
census_data_cleaned.loc[census_data_cleaned[label[0]] == -666666666, label[0]] = np.nan
census_data_new = census_data_cleaned[census_data_cleaned[label[0]].isnull()]
census_data_cleaned = census_data_cleaned[census_data_cleaned[label[0]].notnull()]

Next, create histograms and examine the distribution of each feature to check for normality and skew.

In [ ]:
for i in label + features:
    fig = plt.figure(figsize=(3, 2))
    ax = fig.gca()
    feature = census_data_cleaned[i]
    feature.hist(bins=100, ax=ax, color='gray')
    ax.axvline(feature.mean(), color='blue', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='red', linestyle='dashed', linewidth=2)
    ax.set_title(i)
plt.show()

Some features have a close to normal distribution, while others are skewed right or left. Notably, the label appears to be capped around $250,000.

In [ ]:
census_data_cleaned[label[0]][census_data_cleaned[label[0]] >= 249000].value_counts()

We will drop observations at the income cap to ensure the model doesn't over-predict the number of block groups at this value.

In [ ]:
census_data_cleaned = census_data_cleaned[census_data_cleaned[label[0]] < 250000]

Now, create scatterplots to examine potential associations between the features and the label. We will use a random sample of the data to improve visualization.

In [ ]:
census_data_random_1000 = census_data_cleaned.sample(n=1000, random_state=1)
for i in features:
    fig = plt.figure(figsize=(3, 2))
    ax = fig.gca()
    feature = census_data_random_1000[i]
    lab = census_data_random_1000[label[0]]
    correlation = feature.corr(lab)
    plt.scatter(x=feature, y=lab, s=2, color='gray')
    plt.xlabel(i)
    plt.ylabel('Income')
    ax.set_title('Income vs. ' + i + ': Correlation = ' + str(correlation.round(2)))
plt.show()

Feature relationships with the label vary. As might be expected, the poverty features have the strongest correlations with median household income.

## Preprocess the Data

We will construct a scikit-learn pipeline for simplicity and replicability of preprocessing. To improve model performance, we will apply StandardScaler to scale the numeric features to have a zero-mean and unit variance. We will start by using basic linear regression.

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

Split the data into training and test sets using an 80/20 split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(census_data_cleaned[features], census_data_cleaned[label], test_size=0.2, random_state=1)

## Run and Evaluate the Model

Fit the pipeline to the training set to standardize the features and run the model.

In [ ]:
model = pipeline.fit(X_train, y_train)

Predict the labels for the test set.

In [ ]:
predictions = model.predict(X_test)

Calculate model performance metrics.

In [ ]:
print("MSE:", '{:.0f}'.format(mean_squared_error(y_test, predictions)))
print("RMSE:", '{:.0f}'.format(np.sqrt(mean_squared_error(y_test, predictions))))
print("R-Squared:", '{:.4f}'.format(r2_score(y_test, predictions)))

Create a scatterplot of the predicted and actual values.

In [ ]:
plt.scatter(y_test, predictions, s=2, color='gray')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Median Household Income Predicted vs. Actuals')
plt.show()

The predictions are better than chance, but could potentially be improved. We will run a couple of different models to see if we can improve model performance.

## Run and Evaluate Alternative Models

We will try some ensemble models, which may perform better than basic linear regression because they can model more complex relationships between features and the label.

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor())
])

model = pipeline.fit(X_train, y_train)

predictions = model.predict(X_test)

print("MSE:", '{:.0f}'.format(mean_squared_error(y_test, predictions)))
print("RMSE:", '{:.0f}'.format(np.sqrt(mean_squared_error(y_test, predictions))))
print("R-Squared:", '{:.4f}'.format(r2_score(y_test, predictions)))

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', GradientBoostingRegressor())
])

model = pipeline.fit(X_train, y_train)

predictions = model.predict(X_test)

print("MSE:", '{:.0f}'.format(mean_squared_error(y_test, predictions)))
print("RMSE:", '{:.0f}'.format(np.sqrt(mean_squared_error(y_test, predictions))))
print("R-Squared:", '{:.4f}'.format(r2_score(y_test, predictions)))

As expected, the other models resulted in better performance. We will proceed with the GradientBoostingRegressor because it had slightly better performance than the RandomForestRegresor.

## Apply the Model to New Data

Pickle the file to efficiently save it for later use.

In [ ]:
filename = './predict_household_income.pkl'
joblib.dump(model, filename)

Load the model and the new data to apply it to. Our new dataset will be the records that were missing income data.

In [ ]:
loaded_model = joblib.load(filename)
X_new = census_data_new[features].copy()
results = loaded_model.predict(X_new)
X_new['predicted_income'] = results
X_new.head(3)

## Useful Resources

https://learn.microsoft.com/en-us/training/modules/explore-analyze-data-with-python/5-exercise-visualize-data

https://learn.microsoft.com/en-us/training/modules/train-evaluate-regression-models/7-exercise-optimize-save-models

https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html